Q 1, n step bootstrap


In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(os.path.dirname(module_path))

import itertools as it
import numpy as np

from typing import Iterable, Callable, Mapping, TypeVar, List
from rl.markov_process import TransitionStep
from rl.returns import returns
from rl.function_approx import Tabular
from rl.iterate import last
from collections import defaultdict

from rl.chapter2.simple_inventory_mrp import SimpleInventoryMRPFinite, InventoryState
from rl.monte_carlo import mc_prediction
from rl.td import td_prediction

from rl.distribution import Constant

S = TypeVar('S')

In [8]:

## we will bootstrap frmo 0 to total- n, the rest we will use a normal te
def bootstrap(trans: Iterable[TransitionStep[S]],
                      weight_func: Callable[[int],float],
                          n: int,
                      gamma: float) -> List[Mapping[S,float]]:
    v: Mapping[S,float] = defaultdict(float)
    occurence: Mapping[S,int] = defaultdict(int)
        
    trans = list(trans)
    for i,st in enumerate(trans): ##we are not given a whole episode now
        if i + n <len(trans):
            occurence[st.state] += 1
            G = 0
            for j in range(n):
                ## boot strap
                G += gamma**j * trans[i+j].reward
            G += gamma**n*v[trans[i+n].state]
            v[st.state] += weight_func(occurence[st.state])*(G-v[st.state])

    return v

In [11]:
capacity: int = 2
poisson_lambda: float = 1.0
holding_cost: float = 1.0
stockout_cost: float = 10.0

gamma: float = 0.9

si_mrp: SimpleInventoryMRPFinite = SimpleInventoryMRPFinite(
    capacity=capacity,
    poisson_lambda=poisson_lambda,
    holding_cost=holding_cost,
    stockout_cost=stockout_cost,
)

start = InventoryState(on_hand = 0, on_order = 0)
sample = list(it.islice(si_mrp.simulate_reward(Constant(start)),100000))

res_1 = bootstrap(sample, lambda n: 1./n,3, user_gamma)

In [12]:
res_1

defaultdict(float,
            {InventoryState(on_hand=2, on_order=0): -28.638006406881463,
             InventoryState(on_hand=0, on_order=0): -33.75245851590969,
             InventoryState(on_hand=0, on_order=2): -26.58982951294689,
             InventoryState(on_hand=1, on_order=0): -27.21333956558771,
             InventoryState(on_hand=1, on_order=1): -27.64677548295743,
             InventoryState(on_hand=0, on_order=1): -26.197053736793972})